# SPOTS DETECTOR
## This notebook takes input of raw images(one channel at a time) and outputs spots for that channel over all frames in the image. For detailed analysis of a single frame use other notebooks in this folder.
## Note 
1. The code to generate csv files as output has been commented out and can be used if needed 
2. Pickle files will be outputted to save memory

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


### package for 3d visualization
#from itkwidgets import view                              
#from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
#import skimage

### local new python segmentation functions
import os
import sys
pythonPackagePath = os.path.abspath('../src/')
sys.path.append(pythonPackagePath)
from peak_local_max_3d import peak_local_max_3d 
from gaussian_fitting import fit_multiple_gaussians
from extract_data import extract_data_from_filename
from gaussian_visualization import visualize_3D_gaussians
from gaussian_fitting import check_fitting_error
import matplotlib.pyplot as plt


## Change the following before running the code 
1. input_filename 
2. save_dir 

In [2]:
input_filename = '/Users/apple/Desktop/Akamatsu_Lab/pyLattice_tutorials/test_data/Channel3_complete.tif'

# Specify the directory where you want to save the pickle files
save_dir = '/Users/apple/Desktop/Akamatsu_Lab/pyLattice_tutorials/DataFrames/Channel_3_pkl'

# csv saving directory 
#csv_save_dir = '../DataFrames/Channel_3_csv'

## Change the following before running the code 
1. Intensity for Channel 2(set to 170) and for Channel 3(set to 180)
2. Output Pickle File Name for channel 2 and channel 3

In [ ]:
import time

#original image shape is (t,z,y,x)
image_raw = extract_data_from_filename(input_filename)
#print(image_raw.shape)

#transpose the image such that we get (t,z,x,y)
image_raw = np.transpose(image_raw, axes =(0,1,3,2))

#get the number of frames for our original data for automated analysis for all frames 
frames = image_raw.shape[0]

start_time = time.time()

for frame in range(frames):
    
    print('frame number is', frame)
    
    single_frame_input = image_raw[frame]

    #define threshold: a value(intensity) for the pixel below which all values would be considered noise and dropped 
    #define min_distance: min_distance/2 is the radius within which we will keep the peak with max value/intensity or 
    #if two peaks have the same value they will be kept 
    maximas = peak_local_max_3d(single_frame_input,min_distance=10,threshold=180)
    print('local_maximas detected are', maximas.shape[0])


    #give the expected std dev/radius of our particles for x,y,z 
    sigmaExpected_x__pixels = 2
    sigmaExpected_y__pixels = 2
    sigmaExpected_z__pixels = 4

    sigmas_guesses = []
    for i in range(len(maximas)):
        sigmas_guesses.append([sigmaExpected_z__pixels,sigmaExpected_x__pixels,sigmaExpected_y__pixels])
        
    #last parameter in the fit_multiple_gaussians is similar to min_distance above, we should give half of the 
    #value here of min_distance   
    gaussians, gaussians_popt = fit_multiple_gaussians(single_frame_input,maximas,sigmas_guesses,5)
        
    accumulator = []
    for gaussian in gaussians:

        if(gaussian!=-1):
            amplitude = gaussian[0]

            #print(gaussian)
            mu_x     = int(gaussian[1][1]) ##this is going to be mu_z, previous code [1][0]
            mu_y     = int(gaussian[1][2]) ##need to finalise what this is (x or y) [1][1]
            mu_z     = int(gaussian[1][0]) ##need to finalise what this is (x or y) [1][2]
            ##sigmas will also change due to the above 
            sigma_x  = int(gaussian[2][1]) 
            sigma_y  = int(gaussian[2][2])
            sigma_z  = int(gaussian[2][0])
            accumulator.append(np.array([amplitude,mu_x,mu_y,mu_z,sigma_x,sigma_y,sigma_z]))
            
    accumulator = np.array(accumulator)
    df = pd.DataFrame()
    df['amplitude'] = accumulator[:,0]
    df['mu_x'] = accumulator[:,1]
    df['mu_y'] = accumulator[:,2]
    df['mu_z'] = accumulator[:,3]
    df['sigma_x'] = accumulator[:,4]
    df['sigma_y'] = accumulator[:,5]
    df['sigma_z'] = accumulator[:,6]
    df.head()
    
    error_list = check_fitting_error(single_frame_input,maximas,gaussians,sigmas_guesses)
    
    # Construct the filename
    '''
    filename_csv = f'df_c2_t{frame}.csv'
    file_path_csv = os.path.join(csv_save_dir, filename_csv)
    df.to_csv(file_path_csv)
    '''

    # Construct the filename based on the loop index (time_frame)
    filename_pkl = f'df_c3_t{frame}.pkl'

    # Construct the full file path by joining the directory and filename
    file_path = os.path.join(save_dir, filename_pkl)

    # Save the DataFrame to a pickle file with the specified path
    df.to_pickle(file_path)
end_time = time.time()

print('time taken (seconds)', end_time - start_time)
        


(130, 1, 75, 258, 275)
(130, 75, 258, 275)
frame number is 0
local_maximas detected are 578
10%(58 of 578)
20%(116 of 578)
30%(174 of 578)
40%(232 of 578)
50%(289 of 578)
60%(347 of 578)
70%(405 of 578)
80%(463 of 578)
90%(521 of 578)
100%(578 of 578)
frame number is 1
local_maximas detected are 552
10%(56 of 552)
20%(111 of 552)
30%(166 of 552)
40%(221 of 552)
50%(276 of 552)
60%(332 of 552)
70%(387 of 552)
80%(442 of 552)
90%(497 of 552)
100%(552 of 552)
frame number is 2
local_maximas detected are 567
10%(57 of 567)
20%(114 of 567)
30%(171 of 567)
40%(227 of 567)
50%(284 of 567)
60%(341 of 567)
70%(397 of 567)
80%(454 of 567)
90%(511 of 567)
100%(567 of 567)
frame number is 3
local_maximas detected are 552
10%(56 of 552)
20%(111 of 552)
30%(166 of 552)
40%(221 of 552)
50%(276 of 552)
60%(332 of 552)
70%(387 of 552)
80%(442 of 552)
90%(497 of 552)
100%(552 of 552)
frame number is 4
local_maximas detected are 544
10%(55 of 544)
20%(109 of 544)
30%(164 of 544)
40%(218 of 544)
50%(272 o